## 모델 고도화
- Few-Shot Prompting
- Chain-of-Thought Prompting

In [1]:
import math
import re

from tqdm.notebook import tqdm

from eval import get_eval_data, pointwise_eval
from utils import summarize, get_train_data

In [2]:
PROMPT_BASELINE = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

In [3]:
conv_train = get_train_data()[18]
print(conv_train)

P01: 코로나가 좀 잠잠해지면 해외여행 중에 가고 싶은 곳 있어?
P02: 난 호주 한번 다시 다녀오고 싶엉 키키
P02: 아님... 원래 일본도 가보고 싶었는데 무서워서 안 갈래 키키
P01: 일본은 예전 노 재팬 한 이후로 사실 아직도 웬만하면 일본꺼 안 쓰거든 키키
P01: 나도 일본은 별로고 키키
P01: 나는 하와이 괌 이런 곳 가보고 싶어 키키
P02: 휴양지 좋지 키키
P02: 아아! 베트남 다낭 가보고 싶어~
P02: 좋다던데 키키
P01: 맞아 맞아 나 남편이랑 연애 초기에 남편 첫 직장 입사하고 그 해에 바로 거기로 해외 워크샵 갔는데 진짜 완전 질투나서 키키 연락하다 싸움 키키
P02: 키키 질투 났어? 키키
P02: 아 웃곀 키키
P02: 나중에 우리끼리 가자
P01: 키키 아니 사귄 지 얼마 안 됐는데 아 얼마 안된 건 아닌가 키키
P01: 무튼 연락도 안 되고 전화하면 혼자 신나 있고 난 연락 기다리느라 계속 신경은 거기로만 가있는대 얘는 아니니까 키키
P01: 흔한 그 연인들의 싸움이었지 키키
P01: 무튼 가보고 싶긴 해
P02: 키키 신나 있었어?
P02: 이런 키키
P02: 진짜 떠나고 싶당~
P01: 웅 키키 중딩 때 태권도에서 일본 간 거 말고 첫 해외여행이라고 매우 들떠있었지 키키
P01: 호텔도 엄청 좋은 데로 가고 키키
P01: 첫 직장에 첫 해외니 얼마나 좋았겠어 키키
P02: 엄청 좋지 키키
P02: 우리 대학교 졸업 여행 간 거 기억나?
P02: 태국 키키
P02: 난 그게 첫 해외여행이었어
P01: 기억나지 키키
P01: 내 얼굴에서만 플래쉬 터지고... 키키
P01: 아 재밌었지 웃기고 키키
P01: 태국 가보고 그다음에 간 게 호주 워홀이었던 건가?
P02: 키키 아 맞아~ 플래쉬 키키
P02: 옹옹~ 그 담이 호주였지~
P02: 대담해써 아주키키
P01: 그니까 용기 있었어 키키
P01: 대단해
P01: 돈 열심히 모아서 진짜 스위스 꼭 가봐
P01: 죽어도 여한 없을 것 같은 그런 느낌이야... 키키
P

In [4]:
summary_sample = """최소 대학 생활부터 함께 한 매우 친밀한 사이의 두 사용자가 코로나가 잠잠해졌을 때 방문하고 싶은 해외 여행지에 대해 일상적이고 가벼운 톤으로 대화하고 있습니다.
여행지로는 하와이, 괌, 스위스, 호주, 베트남 다낭 등을 언급하며 남편과의 연락 관련 다툼이나 졸업여행 관련 추억을 회상합니다.
또한 여행 중 발생하는 위험에 대한 우려도 표하고 있으며, 해외여행 시 언어 장벽의 어려움을 인식하고 영어 실력을 향상시키고 싶다는 마음을 가볍게 표현합니다."""

prompt = f"""당신은 요약 전문가입니다. 사용자 대화들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 사용자 대화와 요약 예시입니다.
예시 대화:
{conv_train}
예시 요약 결과:
{summary_sample}
    
아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

In [6]:
models = [
    'claude-3-haiku-20240307',
    # 'gemini-1.5-flash-001',
    # 'gpt-3.5-turbo-0125'
]
scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(5)):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=prompt,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
for model in scores:
    print(scores[model], model)

[6, 9, 5, 8, 7] claude-3-haiku-20240307


In [8]:
models = [
    'claude-3-haiku-20240307',
    'gemini-1.5-flash-001',
    'gpt-3.5-turbo-0125'
]
scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(5)):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=prompt,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
for model in scores:
    print(scores[model], model)

[6, 7, 5, 8, 7] claude-3-haiku-20240307
[8, 6, 6, 9, 9] gemini-1.5-flash-001
[6, 6, 6, 7, 5] gpt-3.5-turbo-0125


Chain-of-thought

In [10]:
summary_sample = """최소 대학 생활부터 함께 한 매우 친밀한 사이의 두 사용자가 코로나가 잠잠해졌을 때 방문하고 싶은 해외 여행지에 대해 일상적이고 가벼운 톤으로 대화하고 있습니다.
여행지로는 하와이, 괌, 스위스, 호주, 베트남 다낭 등을 언급하며 남편과의 연락 관련 다툼이나 졸업여행 관련 추억을 회상합니다.
또한 여행 중 발생하는 위험에 대한 우려도 표하고 있으며, 해외여행 시 언어 장벽의 어려움을 인식하고 영어 실력을 향상시키고 싶다는 마음을 가볍게 표현합니다."""

prompt = f"""당신은 요약 전문가입니다. 사용자 대화들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 대화를 요약할 때는 다음 단계를 따라주세요:

1. 대화 참여자 파악: 대화에 참여하는 사람들의 수와 관계를 파악합니다.
2. 주제 식별: 대화의 주요 주제와 부차적인 주제들을 식별합니다.
3. 핵심 내용 추출: 각 주제에 대한 중요한 정보나 의견을 추출합니다.
4. 감정과 태도 분석: 대화 참여자들의 감정이나 태도를 파악합니다.
5. 맥락 이해: 대화의 전반적인 맥락과 배경을 이해합니다.
6. 특이사항 기록: 대화 중 특별히 눈에 띄는 점이나 중요한 사건을 기록합니다.
7. 요약문 작성: 위의 단계에서 얻은 정보를 바탕으로 간결하고 명확한 요약문을 작성합니다.
각 단계를 수행한 후, 최종적으로 전체 대화를 200자 내외로 요약해주세요.

예시 대화:
{conv_train}
예시 요약 결과:
{summary_sample}
    
아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

In [11]:
models = [
    'claude-3-haiku-20240307',
    'gemini-1.5-flash-001',
    'gpt-3.5-turbo-0125'
]
scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(5)):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=prompt,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
for model in scores:
    print(scores[model], model)

[8, 8, 6, 9, 8] claude-3-haiku-20240307
[8, 8, 5, 9, 7] gemini-1.5-flash-001
[7, 6, 5, 9, 6] gpt-3.5-turbo-0125


In [13]:
prompt = f"""당신은 요약 전문가입니다. 사용자 대화들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 대화를 요약할 때는 다음 단계를 따라주세요:

1. 대화 참여자 파악: 대화에 참여하는 사람들의 수와 관계를 파악합니다.
2. 주제 식별: 대화의 주요 주제와 부차적인 주제들을 식별합니다.
3. 핵심 내용 추출: 각 주제에 대한 중요한 정보나 의견을 추출합니다.
4. 감정과 태도 분석: 대화 참여자들의 감정이나 태도를 파악합니다.
5. 맥락 이해: 대화의 전반적인 맥락과 배경을 이해합니다.
6. 특이사항 기록: 대화 중 특별히 눈에 띄는 점이나 중요한 사건을 기록합니다.
7. 요약문 작성: 위의 단계에서 얻은 정보를 바탕으로 간결하고 명확한 요약문을 작성합니다.
각 단계를 수행한 후, 최종적으로 전체 대화를 200자 내외로 요약해주세요.

아래는 예시 대화와 예시 요약 과정 및 결과 입니다.

예시 대화:
{conv_train}

예시 요약 과정
1. "우리 대학교 졸업 여행 간 거 기억나?"라는 언급과 전반적으로 친밀한 대화 톤을 사용하고 있는 것을 보았을 떄 두 사용자는 오랜 친구 사이로 보입니다.
대화의 시작 부분에서 "코로나가 좀 잠잠해지면 해외여행 중에 가고 싶은 곳 있어?"라고 묻고 있는 것을 보았을 때 코로나 이후 가고 싶은 해외 여행지에 대해 논의하고 있습니다.
따라서 다음과 같이 요약 할 수 있습니다:
최소 대학 생활부터 함께 한 매우 친밀한 사이의 두 사용자가 코로나가 잠잠해졌을 때 방문하고 싶은 해외 여행지에 대해 일상적이고 가벼운 톤으로 대화하고 있습니다.

2. 대화 중 호주, 일본, 하와이, 괌, 베트남 다낭, 스위스, 유럽들이 언급하고 있습니다.
남편의 첫 직장 워크샵, 대학교 졸업 여행, 호주 워킹홀리데이 등의 경험을 이야기하면서 과거 여행 경험을 공유하며 추억을 회상하고 있습니다.
따라서 다음과 같이 요약 할 수 있습니다:
여행지로는 하와이, 괌, 스위스, 호주, 베트남 다낭 등을 언급하며 남편과의 연락 관련 다툼이나 졸업여행 관련 추억을 회상합니다.

3. 소매치기, 여권 분실, 인도에서의 여성 여행자 위험 등을 언급하며 해외 여행의 위험성에 대해 우려를 표현하고 있습니다.
"해외 여행 가면 가이드 안 끼고 가면 영어 실력 엄청 좋은 사람이랑 가는 거 아닐 땐 소통 문제도 좀 곤란할 때가 있는 거 같아"라는 언급과 "왜 영어 공부를 열심히 안 했을까... 후회"라는 표현이 있는 것을 보았을 때 언어 장벽의 어려움을 인식하고 영어 실력 향상에 대한 욕구를 표현합니다.
따라서 다음과 같이 요약 할 수 있습니다:
또한 여행 중 발생하는 위험에 대한 우려도 표하고 있으며, 해외여행 시 언어 장벽의 어려움을 인식하고 영어 실력을 향상시키고 싶다는 마음을 가볍게 표현합니다.

예시 요약 결과
최소 대학 생활부터 함께 한 매우 친밀한 사이의 두 사용자가 코로나가 잠잠해졌을 때 방문하고 싶은 해외 여행지에 대해 일상적이고 가벼운 톤으로 대화하고 있습니다.
여행지로는 하와이, 괌, 스위스, 호주, 베트남 다낭 등을 언급하며 남편과의 연락 관련 다툼이나 졸업여행 관련 추억을 회상합니다.
또한 여행 중 발생하는 위험에 대한 우려도 표하고 있으며, 해외여행 시 언어 장벽의 어려움을 인식하고 영어 실력을 향상시키고 싶다는 마음을 가볍게 표현합니다.
    
아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

In [15]:
models = [
    'claude-3-haiku-20240307',
    'gemini-1.5-flash-001',
    'gpt-3.5-turbo-0125'
]
scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(5)):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=prompt,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
for model in scores:
    print(scores[model], model)

[8, 6, 4, 8, 9] claude-3-haiku-20240307
[8, 5, 5, 9, 7] gemini-1.5-flash-001
[6, 6, 4, 7, 6] gpt-3.5-turbo-0125


In [17]:
models = [
    'claude-3-haiku-20240307',
    # 'gemini-1.5-flash-001',
    # 'gpt-3.5-turbo-0125'
]
scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(5)):
    # for i in tqdm(range(len(get_eval_data()))):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=prompt,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

In [18]:
for model in scores:
    print(scores[model], model)

[8, 6, 6, 8, 9] claude-3-haiku-20240307


In [19]:
import math

for model in scores:
    mean = sum(scores[model]) / len(scores[model])
    variance = sum((x - mean) ** 2 for x in scores[model]) / (len(scores[model]) - 1)
    std_dev = math.sqrt(variance)
    print(f'{model}: {mean} / {round(std_dev, 2)}')

claude-3-haiku-20240307: 7.4 / 1.34


In [20]:
print(get_eval_data()[10])

P01: 더벤티 한 잔만 사면 한잔꽁짜였는데
P01: 쿠폰이 왜 없어졌지
P02: 기간 있는거 아니여?
P01: 그런가봐
P02: 헐
P01: 고흥에 생겨서 좋아했더니 순천이랑도 별갠가봐
P02: 그럼 순천카페 고흥카페 분리된거여
P01: 그런가봐 나 뭐 한거니
P02: 뻘짓한거지 뭐 키키 싸니까 또 열심히 사마셔
P01: 그럴 줄 알았는데 커피를 별로 안 좋아하니까 안 가게 되네
P02: 산책 삼아 다녀오자
P02: 나 커피 사줘
P01: 커피 하루에 한 잔만 마신다며
P02: 그럼 자몽에이드사줘
P01: 예전에는 복숭아 티만 마시더니 변했구나
P02: 나이들면 입맛도 변해
P02: 커피도 믹스커피 하루에 한잔은 마셔야해
P01: 무조건 아메리카노 만 마시던 사람이
P02: 어 그래지더라 믹스 들어가야 힘이나
P01: 나는 믹스텁텁해서 무슨 맛인지 모르겠더라
P02: 그래서 나도 하루에 한잔만 마시는거여
P01: 오늘 ** 언니가 같이 점심 먹자고 했는데 안 갈라고
P02: 왜 갔다오지
P01: 엉클 가자는데 나는 거기 별로였거든
P02: 맛없디?
P01: 비싼 거에 비해서 그냥 그랬어
P02: 그럼 다음에 나랑 가보자
P01: 자기가 사주면 생각해볼게
P02: ...도둑년
P02: 맨날 얻어먹을 생각만 하네
P02: 한번은 니가 사라
P01: 남이 사 주는 거 제일 맛있어
P02: 응 그건 그래 ㅎ
P02: 그럼 오늘 집에만 있을거야?
P01: 아니 이따가 교육 끝나고 언니들이랑 카페 가서 차 한 잔 할라고
P02: 그래 잘갔다와


In [21]:
PROMPT_BASELINE = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

summary = summarize(
    conversation=get_eval_data()[10],
    prompt=PROMPT_BASELINE,
    model='claude-3-haiku-20240307'
)
print(summary)

1. 더벤티 카페에서 한 잔 공짜였던 쿠폰이 없어져서 아쉬워했다.
2. 커피 취향이 변화하여 믹스커피와 자몽에이드를 마시게 되었다.
3. 친구와 함께 카페에 가기로 하고, 친구가 사주기를 바랐다.


In [ ]:
summary = summarize(
    conversation=get_eval_data()[10],
    prompt=prompt,
    model='claude-3-haiku-20240307'
)
print(summary)